# Módulo 4: Modelo de clasificación QSAR (paso a paso)

En este módulo desarrollaremos nuestro primer modelo QSAR, concretamente un modelo para la toxicidad aguda en lombrices de tierra. Dado que desarrollar un modelo QSAR implica un flujo de trabajo con varios pasos y este es tu primer modelo, este módulo será extenso, ya que exploraremos cada paso cuidadosamente. Por ello, el flujo de trabajo se divide en diferentes lecciones, y la práctica en Python correspondiente está separada en distintos archivos Jupyter Notebook. Como recordatorio, en este curso el flujo de trabajo para desarrollar un modelo QSAR se divide en las siguientes partes:

- Parte 1: Obtención y depuración de datos
- Parte 2: Cálculo de descriptores moleculares
- Parte 3: División entre entrenamiento y prueba, y estandarización
- Parte 4: Selección de descriptores
- Parte 5: Desarrollo y optimización del modelo
- Parte 6: Predicción y dominio de aplicabilidad

# Parte 6: Predicción y dominio de aplicabilidad

En esta lección aprenderemos cómo realizar una predicción con nuestro modelo optimizado y utilizaremos diferentes métodos para evaluar si la sustancia predicha se encuentra dentro del espacio químico del modelo (es decir, dentro de su dominio de aplicabilidad).

Primero, leeremos un modelo optimizado y comprobaremos que es el modelo que queremos utilizar [Sección 1].

En segundo lugar, realizaremos una predicción para una molécula de ejemplo: la cafeína [Sección 2].

A continuación, exploraremos tres métodos distintos para evaluar el dominio de aplicabilidad y los aplicaremos a la molécula de ejemplo:

* Leverages [Sección 3],

* Similitud Tanimoto-Jaccard [Sección 4],

* Distancias euclidianas [Sección 5].

Finalmente, aplicaremos el procedimiento completo a una lista de moléculas [Sección 6].

The third step is to prepare the files that we are going to use along the lesson. In this case, files will be requested later, but remember to upload them to be able to read. (Check previous lessons if need help to upload files)



## Sección 1: Preparación del modelo a usar para la predicción

Para realizar una predicción QSAR, necesitamos tener un **modelo QSAR**.
Por lo tanto, vamos a **recuperar nuevamente el modelo seleccionado**.
Vamos a utilizar la misma función que empleamos en la lección anterior para **entrenar un modelo** a partir del nombre de los archivos de descriptores (entrenamiento y prueba) y del algoritmo configurado.

Ejecuta la siguiente celda para **definir la función**.

In [ ]:
#Importa las librerias necesarias
import pandas as pd
from sklearn.metrics import recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import cross_validate


results={}
def check_classification_model(model,train_file,test_file,name,save=True):

    #Carga el dataset de entrenamiento y test y crea dataframes con los descriptores y la variable dependiente
    train_dataset = pd.read_csv(train_file,sep=',')
    X_train = train_dataset.iloc[:, 2:]
    Y_train = train_dataset["y"]

    test_dataset = pd.read_csv(test_file,sep=',')
    X_test = test_dataset.iloc[:, 2:]
    Y_test = test_dataset["y"]

    #Ajusta el modelo (solo con el train)
    model.fit(X_train,Y_train)

    #Predice el train y calcula las metricas
    Y_train_pred = model.predict(X_train)
    tr_Accuracy=round(accuracy_score(Y_train,Y_train_pred),2)
    tr_Sensitivity=round(recall_score(Y_train,Y_train_pred),2)
    tr_Specificity=round(recall_score(Y_train,Y_train_pred,pos_label=0),2)
    tr_Precision=round(precision_score(Y_train,Y_train_pred),2)
    tr_cm=confusion_matrix(Y_train,Y_train_pred)

    #Predice el test y calcula las metricas
    Y_test_pred = model.predict(X_test)
    ts_Accuracy=round(accuracy_score(Y_test,Y_test_pred),2)
    ts_Sensitivity=round(recall_score(Y_test,Y_test_pred),2)
    ts_Specificity=round(recall_score(Y_test,Y_test_pred,pos_label=0),2)
    ts_Precision=round(precision_score(Y_test,Y_test_pred),2)
    ts_cm=confusion_matrix(Y_test,Y_test_pred)

    #Aplica la validacion cruzada y guarda solo la exactitud(accuracy) del test
    cv_results = cross_validate(model, X_train, Y_train,scoring='accuracy',return_train_score=True)
    cv_Accuracy_mean=round(cv_results['test_score'].mean(),3)
    cv_Accuracy_dev=round(cv_results['test_score'].std(),3)

    if save:
      results.update({name:[tr_Accuracy,tr_Sensitivity,tr_Specificity,tr_Precision,
            ts_Accuracy,ts_Sensitivity,ts_Specificity,ts_Precision,
            cv_Accuracy_mean,cv_Accuracy_dev]})

    #Muestra la matriz de confusion y las metricas calculadas
    print('\t\tTRAIN\t\t\t\tTEST')
    print('\t\tNeg.\tPos.\t\t\tNeg.\tPos.')
    print('\tNegative\t'+str(tr_cm[0][0])+'\t'+str(tr_cm[0][1])+'\t\tNegative\t'+str(ts_cm[0][0])+'\t'+str(ts_cm[0][1]))
    print('\tPositive\t'+str(tr_cm[1][0])+'\t'+str(tr_cm[1][1])+'\t\tPositive\t'+str(ts_cm[1][0])+'\t'+str(ts_cm[1][1]))
    print('Accuracy:\t',tr_Accuracy,'\t\t\t\t',ts_Accuracy)
    print('Sensitivity:\t',tr_Sensitivity,'\t\t\t\t',ts_Sensitivity)
    print('Specificity:\t',tr_Specificity,'\t\t\t\t',ts_Specificity)
    print('Precision:\t',tr_Precision,'\t\t\t\t',ts_Precision)
    print('Cross validation test accuracy:',cv_Accuracy_mean,'+/-',cv_Accuracy_dev)

    return [tr_Accuracy,tr_Sensitivity,tr_Specificity,tr_Precision,
            ts_Accuracy,ts_Sensitivity,ts_Specificity,ts_Precision,
            cv_Accuracy_mean,cv_Accuracy_dev]



Aplica la función al modelo que has seleccionado y realiza las predicciones.

In [ ]:
#Importa el algoritmo seleccionado
from ______________  

#Inicializa el estimador con los hiperparametros seleccionados
estimator = _______________  

check_classification_model(estimator, 
                           _________, 
                           ___________________,  
                           'Selected_model')  

## Sección 2: Predicción de la actividad de la cafeína frente a lombrices de tierra

Comencemos con la cafeína. Tu primera tarea es obtener el SMILES de la cafeína.

Una vez lo tengas, aplica el código en Python que aparece a continuación para visualizar la molécula a partir del SMILES utilizando RDKit (creando un objeto mol).

In [ ]:
#Importa la Chem de RDKit


#Asigna el SMILES de la molecula y obtén el objeto mol
smiles= _________________  
mol = _________________    

mol

El siguiente paso es calcular los descriptores, retomando lo que hiciste en la parte 2.
En este caso, no necesitas calcular todos los descriptores, sino solo aquellos que estás utilizando en tu modelo.
Por lo tanto, calcular únicamente los descriptores necesarios es una alternativa más eficiente.

Sin embargo, para simplificar el código en este punto, puedes reutilizar el código de la parte 2 para calcular todos los descriptores y guardarlos en un DataFrame.
Los pasos para esto son:

1. El código utilizado en la parte 2 emplea un DataFrame para introducir la lista de SMILES, así que debes crear un DataFrame con la columna "SMILES".

1. Calcular los descriptores de RDKit e incluirlos en el mismo DataFrame.

1. Calcular los descriptores de Mordred en un DataFrame diferente y concatenarlo al anterior.

1. Seleccionar únicamente las columnas correspondientes a los descriptores seleccionados y guardarlas en un DataFrame diferente (llámalo X_caf).

Finalmente, deberías obtener un DataFrame X_caf con las columnas de solo los descriptores (puedes obtener los nombres de X_train o X_test).
Ten en cuenta que el orden de las columnas es importante, ya que el código que genera y utiliza el modelo no leerá los nombres, sino que asumirá que están en el mismo orden.

Notas:

* Evita usar "descriptors" como nombre de variable porque sobrescribirá la lista importada de descriptores de Mordred.

* Puedes utilizar directamente la columna 'SMILES' como lista de SMILES o guardarla como lista aparte.

In [ ]:
#Importa los módulos y funciones necesarios (pandas, mordred y rdkit)



#Crea un dataframe con una sola columna que se llama SMILES e incluye una lista con los SMILES
df_caf=___________


#Calcula los descriptores de RDKit


#Concatena los resultados a df_caf (los SMILES) en un nuevo dataframe df_caf_desc
df_caf_desc = pd.concat(________________________)


#Calcula los descriptores de Mordred



#Concatena los resultados a df_caf_desc en un nuevo dataframe df_caf_desc2
df_caf_desc2 =


#Muestra el tamaño del dataframe para asegurarte de que todos los descriptores están incluidos
print("\nShape of the final df:",df_caf_desc.shape)


#Carga el dataset de entrenamiento o de validación para obtener el nombre de los descriptores
train_database = pd.read_csv(___________)


#La lista de descriptores debe ser todas las columnas excepto las dos primeras
descs =_______________

X_caf = df_calc_df2[descs]


#Visualiza los descriptores calculados
X_caf

Revisa los valores de los descriptores obtenidos para tu molécula objetivo en el DataFrame anterior.
En algunos casos, el cálculo de descriptores puede proporcionar valores no numéricos que no son útiles (como errores de cálculo o falta de parámetros para ciertos elementos).
(Si ves valores como `True` y `False`, no son un problema, ya que se trata de variables binarias que el modelo interpreta como 1 y 0).

No vamos a tener esto en cuenta aquí por simplicidad, pero la presencia de valores no numéricos, especialmente si representan un porcentaje significativo, es una señal de mala calidad en la predicción y puede implicar que no se pueda usar esa molécula en el modelo.
Como hicimos con el conjunto completo, este problema se puede evitar utilizando un imputador kNN, que emplea el promedio de los valores de unas pocas moléculas vecinas del conjunto de datos original.

De hecho, dado que entrenamos el modelo con una versión imputada y estandarizada de los descriptores,
debemos seguir el mismo procedimiento en lugar de usar los descriptores sin procesar.
Por tanto, para poder calcular una predicción, necesitamos comprobar los descriptores de la molécula de predicción, rellenar los huecos y estandarizarlos usando el mismo método.

Para comenzar este proceso, necesitamos recuperar los valores originales (sin procesar) de los descriptores obtenidos para el conjunto de entrenamiento (que fueron guardados en un archivo CSV después de la división).
Abre en un DataFrame el CSV con los descriptores obtenidos antes de la imputación y estandarización del archivo de entrenamiento, y filtra las columnas correspondientes a los descriptores seleccionados para el modelo que vamos a usar.
Llámalo `database_reduced`.




In [ ]:
#Obtén los valores preprocesados para entrenar el imputer/scaler
train_database_noscaled = ______________________


database_reduced = train_database_noscaled[____________]

 
print(database_reduced.columns)

Además, necesitamos repetir la **estandarización** de los descriptores con el conjunto de datos original, como hicimos anteriormente.
Pero en este caso, debemos **restringir el escalado a los descriptores que vamos a reescalar** (usando el DataFrame `database_reduced`).

En la siguiente celda puedes utilizar la función imputer_scaler (similar a la definida previamente en la lección 2.5)
para realizar la **imputación y el escalado** de una o más moléculas objetivo.

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler


def imputer_scaler(df_target, df_reference):
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=3, weights="uniform")
    scaler = StandardScaler()

    columns_descriptors = list(df_reference.columns)

    reference_descriptors_matrix = df_reference[columns_descriptors]

    imputer.fit(reference_descriptors_matrix)

    target_descriptors_matrix = df_target[columns_descriptors]

    imputed_ref_matrix = imputer.transform(reference_descriptors_matrix)
    imputed_matrix = imputer.transform(target_descriptors_matrix)
    scaler.fit(imputed_ref_matrix)

    imputed_scaled_matrix = scaler.transform(imputed_matrix)
    df_imputed_scaled = pd.DataFrame(imputed_scaled_matrix, columns = columns_descriptors)

    return df_imputed_scaled

#Completa con las matrices de descriptores requeridas
X_caf_scaled = imputer_scaler(_______________)

X_caf_scaled

Después de ejecutar tu código para ajustar y comprobar el modelo, realizar una predicción se hace de la misma manera que cuando evaluamos el entrenamiento y el test.

Ten en cuenta que el resultado será una **lista con todas las predicciones**, incluso si solo se ha hecho una.
¿Cuál es la **predicción para la cafeína**?


In [ ]:
#Predice la toxicidad de la cafeina con el modelo entrenado
Y_caf_pred =   _______


#Muestra los resultados de la predicción
Y_caf_pred

### **Q1.** ¿Cuál es la toxicidad predicha para la cafeina?

¿Cómo de seguros estamos de esta predicción?
Un primer enfoque es comprobar la probabilidad asignada por el propio algoritmo de aprendizaje automático.
Utiliza el método `predict_proba` para obtener la probabilidad de que sea negativa (primer valor) y positiva (segundo valor).

In [ ]:
#Obtén la probabilidad de que la cafeina sea positiva para la toxicidad
Y_caf_prob = ____________


#Muestra la probabilidad
Y_caf_prob

**Nota:** Si has seleccionado una máquina de vectores de soporte (SVM) como algoritmo, esto no es posible por defecto.
Existe una forma de habilitar el cálculo de probabilidades (que debe activarse durante el ajuste del modelo), pero no es necesario para continuar.

## Dominio de Aplicabilidad de la Cafeína en el Modelo de Lombrices de Tierra

Se ha realizado una predicción, pero ¿podemos confiar en ella?
Además de evaluar la calidad general del modelo, necesitamos comprobar si la molécula que estamos analizando se encuentra dentro del espacio químico considerado por nuestro modelo.
Para ello, medimos el dominio de aplicabilidad.

Existen varios métodos para hacerlo, como:

* Leverage

* Distancia euclidiana

* Similitud de Tanimoto

* Estimación de Densidad por Núcleo (Kernel Density Estimation)



## Sección 3: Índice de Tanimoto

El índice de Tanimoto es una medida de similitud molecular basada en las huellas moleculares (fingerprints).
Por lo tanto, lo primero que necesitamos es obtener una huella molecular.
Una fingerprint es un identificador formado por una cadena binaria que indica la presencia o ausencia de determinados grupos o características en una molécula.

No existe un único conjunto de características, por lo que hay varios tipos de fingerprints disponibles.
En este caso, vamos a utilizar la huella MACCS Keys.

Se puede calcular utilizando el módulo rdMolDescriptors de RDKit
(consulta la documentación aquí: https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html).

Obtén la huella MACCS de la cafeína.


In [ ]:
from rdkit.Chem import rdMolDescriptors

#Obténe el fingerprint de la cafeina usando el objeto mol creado antes
fingerprint = __________________


# Keep the prints to check your work
print("Fingerprint should be an ExplicitBitVect object:",fingerprint)
print("This object can be expanded to a list of 0/1 values:",list(fingerprint))


Como puedes ver, la huella molecular (fingerprint) es un objeto binario por motivos de eficiencia, pero se puede convertir en una lista para comprobar individualmente los valores 0 y 1.

Ahora necesitamos comparar esta fingerprint con las del conjunto de entrenamiento. Vamos a utilizar el coeficiente de similitud de Jaccard-Tanimoto, que se calcula como:

$$S_{J-T} = \frac{P_{both}}{P_{both}+P_{1}+P_{2}} $$
donde P representa el número de características que son positivas en ambas moléculas, solo en la primera y solo en la segunda (${P_{both}}$, $P_{1}$, y $P_{2}$, respectivamente).

Como prueba, vamos a calcular el índice de similitud de la cafeína con el ácido butírico y la nicotina, utilizando la fórmula mencionada anteriormente.
Ten en cuenta que la forma más sencilla de operar con los 0s y 1s del índice es convertirlo a una lista previamente.



In [ ]:
but_smiles=  Chem.MolFromSmiles("O=C(O)CCC")
butiric = rdMolDescriptors.GetMACCSKeysFingerprint(but_smiles)
nico_smiles = ___________
nicotine = ____________
lbutiric = list(butiric)
lnicotine = list(nicotine)
lfingerprint = list(fingerprint)

p_both = 0
p_1 = 0
p_2 = 0
for i,j in enumerate(fingerprint):
    if  lfingerprint[i]==__  and lnicotine[i]==__:
        p_both = p_both+1
    elif lfingerprint[i]==__ and lnicotine[i]==__:
        p_1 = p_1+1
    elif lfingerprint[i]==__ and lnicotine[i]==__:
        p_2 = p_2+1

sim_nico = p_both / (p_both+p_1+p_2)
p_both = 0
p_1 = 0
p_2 = 0
for i,j in enumerate(fingerprint):
    if  lfingerprint[i]==__ and lbutiric[i]==__:
        p_both = p_both+1
    elif lfingerprint[i]==__ and lbutiric[i]==__:
        p_1 = p_1+1
    elif lfingerprint[i]==__ and lbutiric[i]==__:
        p_2 = p_2+1

sim_buti = p_both / (p_both+p_1+p_2)

# Keep the prints to check your work
print("Fingerprint of caffeine:",list(fingerprint))
print("Fingerprint of nicotine:",list(nicotine))
print("Fingerprint of butiric acid:",list(butiric))
print("Similarity index for nicotine (should be 0.386):",round(sim_nico,3))
print("Similarity index for butiric acid (should be 0.070):",round(sim_buti,3))

Por otro lado, RDKit tiene una función llamada `FingerprintSimilarity` que calcula directamente el índice de similitud a partir del vector binario.
Utiliza esta función y comprueba que devuelve los mismos valores.

In [ ]:
from rdkit import DataStructs

sim_nico = DataStructs.FingerprintSimilarity(fingerprint,nicotine)
sim_buti = DataStructs.FingerprintSimilarity(fingerprint,butiric)

print("Similarity index for nicotine (should be 0.386):",round(sim_nico,3))
print("Similarity index for butiric acid (should be 0.070):",round(sim_buti,3))


Para aplicar esto y comprobar el espacio químico de una base de datos, necesitamos calcular el índice de similitud de nuestra molécula objetivo con cada molécula del conjunto de entrenamiento.
Para cada una de ellas, obtendremos su fingerprint y calcularemos el índice.

Finalmente, para que la cafeína esté dentro del dominio de aplicabilidad, debe ser similar al menos a una de nuestras moléculas.
Por tanto, solo necesitamos centrarnos en la más similar (el valor más alto).

Así que no es necesario almacenar todos los valores, sino únicamente mantener el valor máximo y mostrarlo al final.

In [ ]:
#Importa los módulos necesarios


#Para evitar errores, mejor incluye de nuevo el SMILES de la cafeina y vuelve a obtener el objeto mol
smiles=__________________
mol =__________
target_fingerprint = ____________________


#Usa la columna SMILES del dataframe train_database abierto antes
smi_train = train_database['SMILES']

#Define la variable para guardar el maximo y asignale un valor bajo (0.0)
max_sim = 0.00
#Itera sobre los SMILES del dataset de entrenamiento
for i in ____________________
    #Obtén el objeto mol y el fingerprint

    train_fingerprint =  ___________________

    #Calcula el score (con la funcion FingerprintSimilarity)
    sim_score = rdkit.DataStructs.FingerprintSimilarity(_______________)
    #Actualiza el valor si el nuevo es mayor
    if sim_score > max_sim:
        max_sim = sim_score

print('Maximum similarity index:',round(max_sim,3))

El valor de similitud considerado como incluido en el dominio de aplicabilidad depende del tipo de fingerprint utilizado.
Por ejemplo, para las huellas MACCS, un valor aceptable es 0.573, lo que corresponde a una similitud superior al 95 %.

Puedes encontrar más información en: http://rdkit.blogspot.com/2013/10/fingerprint-thresholds.html

### **Q2.** ¿Cuál es el valor máximo del índice de similitud de Tanimoto-Jaccard para la cafeína en tu conjunto de datos?

## Sección 4: Leverage

En estadística, el leverage es una medida estadística de cuán diferente es cada compuesto respecto al grupo, basándose en los valores de los descriptores.
A continuación encontrarás una función con el cálculo de esta medida, que es matemáticamente algo compleja y no vamos a analizar en detalle.

In [ ]:
import numpy as np

def leverage(X_train,X_target):
        X_train = X_train.astype(float)


        tranpX = np.transpose(X_train)
        xtx = np.dot(tranpX, X_train)
        invxtx = np.linalg.inv(xtx)


        leverages_target = []

        for idx, row in X_target.iterrows():
            transposed_row = np.transpose(row)
            leverage = np.dot(np.dot(row, invxtx), transposed_row)
            leverages_target.append(leverage.round(4))

        return leverages_target



Además de las diferencias estructurales, los valores de leverage dependen en gran medida del número de moléculas incluidas en el conjunto de entrenamiento y del número de descriptores.
Por lo tanto, el valor utilizado para evaluar si una molécula es lo suficientemente similar como para estar incluida en el dominio de aplicabilidad depende de esos dos factores, y se calcula como: 3 × (número de descriptores / número de moléculas)

In [ ]:
#Crea un DataFrame con los descriptores de la base de datos de entrenamiento
X_train_scaled=  _____________________

#Calcula el límite de leverage
p = X_train_scaled._______  #número de descriptores
n = X_train_scaled.__________  #número de moléculas
warning_leverage = 3*(p/n)

leverages = leverage(X_train_scaled,X_caf_scaled)

leverages_in_out = []
for value in leverages:
    if value < warning_leverage:
            leverages_in_out.append('IN')
    else:
            leverages_in_out.append('OUT')

## Sección5: Distacnia Euclideana

Otra medida de similitud entre moléculas que puede utilizarse para evaluar el dominio de aplicabilidad es la distancia euclidiana. La distancia euclidiana es la medida de distancia comúnmente utilizada en geometría, pero en este caso se extiende a N dimensiones, donde cada dimensión representa uno de los descriptores utilizados en el modelo. La fórmula es

$d(p,q) = \sqrt{(p_1+q_1)^2+(p_2+q_2)^2+...+(p_n+q_n)^2} $, para n dimensiones, donde $p_i$ y $q_i$ son los valores estandarizados del descriptor i para cada molécula.

Como la magnitud de las distancias euclidianas depende de los descriptores, no existe un valor de referencia fijo.
Por ello, necesitamos comparar la distancia de nuestra molécula con el espacio químico en función de las distancias entre las moléculas del conjunto de entrenamiento.

Podemos escribir código para calcular esta distancia y recorrer el conjunto de entrenamiento, como hicimos con la similitud de Tanimoto,
pero en este caso vamos a aprovechar una función de sklearn que calcula directamente la distancia euclidiana por pares para todo el conjunto de datos:
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html

En primer lugar, calculamos la distancia euclidiana entre X_train y sí mismo.

Esta función proporciona todos los valores de la matriz de distancias por pares como una lista de listas, ya que permite calcular distancias para más de una predicción al mismo tiempo.
Como con Tanimoto, no necesitamos todos los valores, sino uno solo que nos sirva para justificar si nuestra molécula está dentro del dominio de aplicabilidad.
En este caso, nos interesa la máxima distancia entre moléculas del conjunto de entrenamiento.

Pista:
Una forma sencilla de calcular el valor máximo de la matriz (lista de listas) es aplanarla a una sola lista usando el método flatten: `list_of_lists.flatten()`



In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

tr_distances = euclidean_distances(X_train_scaled,X_train_scaled)


max_ed= max(tr_distances.flatten())

print('The maximum value of the Eucledian distance in the training set is ', max_ed)


El siguiente paso es calcular la distancia euclidiana máxima entre nuestra molécula objetivo, la cafeína, y el conjunto de entrenamiento,
utilizando la misma función (`euclidean_distances`).
Si esta distancia es menor que la máxima distancia entre moléculas del conjunto de entrenamiento, entonces nuestra molécula está dentro del dominio de aplicabilidad.

Incluye en tu código no solo el cálculo de la distancia euclidiana máxima,
sino también una variable llamada `AD_eucledian` con valor `YES` o `NO` dependiendo del resultado.

In [ ]:
#Importa los módulos necesarios
from sklearn.metrics.pairwise import euclidean_distances

#Las distancias se calculan de la misma manera, pero combinando los dataframes de objetivo y entrenamiento
caf_distances = _______________________

max_ed_caf= _________________________

if max_ed_caf <= max_ed:
    AD_eucledian = _________ #Asigna IN o OUT
else:
    AD_eucledian = __________ #Asigna IN o OUT

print('The maximum value of the Eucledian distance between caffeine and the training set is ', max_ed_caf)
print('Is caffeine in the applicability domain of your model?',AD_eucledian)

## Sección 6: Ejercicio con multiples moléculas

Predice la toxicidad de las siguientes moléculas utilizando el modelo de toxicidad aguda en lombrices de tierra e incluye si están dentro del dominio de aplicabilidad, según el índice de Jaccard-Tanimoto:

* Nombre: Fenol

* CAS: 50-78-2

* SMILES: COCO

* Nombre: Nicotina

* Nombre: Estreptoquinasa

* CAS: 9002-01-1



 ### Paso 1: Genera un DataFrame con una única columna llamada 'SMILES' y complétalo con los SMILES de tus moléculas objetivo.

In [ ]:
#ESCRIBE AQUÍ TU CÓDIGO

### Paso 2: Comprueba el modelo:
* Regenera el modelo

* Recupera X_train con los descriptores sin escalar del conjunto de entrenamiento

* Recupera la lista de descriptores seleccionados como `sel_descriptors`

(Esto realmente no es necesario si ya tienes las variables de secciones anteriores, pero es recomendable repetirlo para asegurarse de que estamos accediendo al modelo correcto).


In [ ]:
estimator = ___________________________
metrics = check_classification_model(estimator,
                           ,
                           ,
                           'Final')

train_database = pd.read_csv(______________)  #Conjunto de entrenamiento no escalado

descriptors_df = pd.read_csv(_____________) # Descriptores seleccionados
sel_descriptors =_____________

#Reduce el juego de datos de entrenamiento a los descriptores seleccionados
train_database = train_database[sel_descriptors]
train_database_scaled = descriptors_df[sel_descriptors]


### Paso 3: Calcula y procesa los descriptores seleccionados para todos los compuestos objetivo.

Revisa y adapta el código de la Sección 2.
En este caso, los SMILES deben leerse desde df_target y los descriptores seleccionados provienen de la celda anterior.
Puedes dividir el código en varias celdas si lo prefieres.

In [ ]:
#Calcula los descriptores de RDKIT


#Calcula los descriptores de Mordred


#Combina los dos dataframes en uno nuevo
df_target_desc=_______


#Muestra el tamaño del dataframe para asegurarte de que todos los descriptores están incluidos
print("\nShape of the final df:",df_target_desc.shape)

#Reduce el dataframe a los descriptores seleccionados
X_target = df_target_desc[sel_descriptors]

#Ejecuta la función imputer_scaler para obtener los descriptores escalados
X_target_scaled = imputer_scaler___________________




print(X_target)
print(X_target_scaled)

### Paso 4: Lleva a cabo las predicciones y guárdalas como una lista.

Recuerda que el método `predict` lee una tabla completa, por lo que puedes hacer todas las predicciones a la vez.

In [ ]:
#ESCRIBE AQUÍ TU CÓDIGO

### Paso 5: Calcula el dominio de aplicabilidad.

  Aplica **el índice de Jaccard-Tanimoto** y obtén una **lista con valores 'YES'/'NO'** que indiquen si cada molécula está dentro del **dominio de aplicabilidad** (AD).

 * Índice de Jaccard-Tanimoto

In [ ]:
#@title Use tanimoto
threshold = 0.573


#Obtén la lista de SMILES de los datasets
smi_train = descriptors_df['SMILES']
smi_target = _____


AD_tanimoto=[]
for smi in smi_target:
    target_fingerprint = ________

    for i in smi_train:
        train_fingerprint = _____
        sim_score = ______
        #Comprueba si el score es obtenido es el mayor


    #Muestra la mayor similitud obtenida
    print('Maximum similarity index:',round(max_sim,3))
    #Comáralo con el umbral para añadir YES o NO a la lista

#Muestra la lista de resultados
AD_tanimoto

Presenta tus resultados como un DataFrame con las columnas "SMILES", "Prediction", "Tanimoto".

In [ ]:
final_dataframe = pd.DataFrame(df_target['SMILES'],columns=['SMILES'])
final_dataframe['Prediction'] = ____
final_dataframe['Tanimoto'] = ______


final_dataframe